In [1]:
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import string

from konlpy.tag import Mecab
import numpy as np

C:\Users\user\anaconda3\envs\konlpy\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


한국어 코퍼스인 네이버 영화리뷰 데이터를 사용하여 word2vec 모델을 훈련시킵니다. 전처리 과정을 마친 훈련 데이터셋을 이용할 것이며 전처리 과정은 해당 링크를 참고하면 됩니다.

In [2]:
df = pd.read_csv("train_for_korean.csv", encoding="utf-8-sig")

In [3]:
df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터 보고 초딩 영화 줄 오버 연기조차 가볍지 않구나,1
2,10265843,너무 재밓었다 그래서 보는 것을 추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...,1


In [4]:
df = df[["document"]]

In [5]:
df.head()

,document
0,아 더빙 진짜 짜증나네요 목소리
1,흠포스터 보고 초딩 영화 줄 오버 연기조차 가볍지 않구나
2,너무 재밓었다 그래서 보는 것을 추천한다
3,교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4,사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...


해당 데이터셋에는 결측치가 있을 뿐만 아니라 공백으로 이루어진 문자열도 있습니다. 그렇기에 해당 데이터들은 지웁니다.

In [6]:
df = df.dropna(axis=0)
space_idx = []
for i in range(len(df)):
    if str.isspace(df.iloc[i, 0]) == True:
        space_idx.append(i)
df = df.drop(df.index[[space_idx]])

C:\Users\user\anaconda3\envs\konlpy\lib\site-packages\pandas\core\indexes\base.py:4308: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


Word2Vec을 훈련시키기 위해 tensor의 모양을 변형시킨 후 각 문장을 토큰화 합니다

In [7]:
data = df.to_numpy()
print(data.shape)

(149646, 1)


In [8]:
data = data.reshape(-1)
print(data.shape)

(149646,)


In [9]:
m = Mecab("C:\mecab\mecab-ko-dic")
print(m.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))

['열심히', '코딩', '한', '당신', ',', '연휴', '에', '는', '여행', '을', '가', '봐요']


In [10]:
for i in range(len(data)):
    data[i] = m.morphs(data[i])

In [21]:
data[1]

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나']

본격적으로 Word2Vec을 훈련시킵니다. 각 단어는 100차원 벡터 공간으로 표현되며 윈도우 사이즈가 5인 Skip Gram으로 훈련시킵니다. 이 때 단어의 최소 출현 수가 5회 이상인 단어들에 대해서만 학습을 진행합니다.

In [15]:
model = Word2Vec(sentences=data, vector_size=100, window=5, min_count=5, workers=4, sg=1)
# size : 임베딩 벡터 사이즈
# window : 윈도우 사이즈
# min_count : 단어 최소 빈도 수 제한
# workers : 학습을 위한 프로세스 수
# sg : 0 = CBOW, 1 = Skip-gram

학습시킨 Word2Vec 모델이 "학교"라는 단어와 가장 유사하다고 판단한 단어들을 출력합니다. 더불어 "학교"라는 단어가 어떻게 표현됐는지 벡터를 확인합니다.

In [18]:
model_result = model.wv.most_similar("학교")
print(model_result)

[('학년', 0.8192612528800964), ('초등', 0.8187326788902283), ('중학교', 0.7957491278648376), ('고등학교', 0.7774748206138611), ('대학교', 0.7695490121841431), ('대학', 0.7577673196792603), ('EBS', 0.7564696073532104), ('저학년', 0.7461627125740051), ('동아리', 0.7402120232582092), ('다닐', 0.7393215298652649)]


In [20]:
print(model.wv["학교"])

[-0.14919935  0.37061924 -0.5168925   0.45035076  0.7908818  -0.29296067
  0.23340262  0.21285623  0.09351742 -0.19124721 -0.29800478 -0.08264505
 -0.3013297  -0.04739086  0.06156342 -0.1707079   0.572063   -0.52907497
  0.25912708 -0.5796304  -0.13098337 -0.2906048   0.38956308 -0.4195059
  0.09766378 -0.99081033 -0.5212223  -0.03514219  0.7170484  -0.03492802
 -0.24713577  0.31983596  0.07705984  0.25733083 -0.2954667   0.45152187
  0.06174426 -0.7743571   0.24291737 -0.36259735 -0.17902227 -0.0737453
  0.3603414  -0.16491678 -0.01550239  0.04524917 -0.1795484   0.3362554
 -0.09584425  0.42342502  0.0363703  -0.6856256   0.15439722  0.15333527
 -0.52088505  0.17987941  0.14759825 -0.05025753 -0.45552543  0.2990967
  0.01507362 -0.31144735  0.10149104 -0.35816777  0.07904858  0.05298008
  0.2261061   0.21521099 -0.71064913 -0.26958907 -0.07876834  0.284706
 -0.05561232 -0.00950367  0.27085766 -0.3528704  -0.352523    0.4248009
 -0.00305147  0.16771983 -0.43436423  0.2658655  -0.322221